# Задание:

Обучить нейросеть на датасете  CIFAR 10. Это датасет из 60000 изображений разделенных на 10 классов. А также оцените качество работы этой модели.


In [1]:
import torch  # Основная библиотека PyTorch для работы с тензорами и вычислениями на GPU/CPU
import torch.nn as nn  # Модуль для построения нейронных сетей (слои, активации и т.п.)
import torch.optim as optim  # Модуль для оптимизаторов — алгоритмов обновления весов сети
import torchvision  # Библиотека с готовыми датасетами, моделями и преобразованиями для компьютерного зрения
import torchvision.transforms as transforms  # Модуль для преобразований изображений (аугментации, нормализация)
import matplotlib.pyplot as plt  # Библиотека для визуализации графиков и изображений

In [2]:

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # nn.Conv2d: сверточный слой с параметрами:
        # 3 входных канала (RGB), 32 выходных фильтра, ядро 3x3, паддинг 1 (чтобы сохранить размер)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        # nn.MaxPool2d: операция подвыборки с ядром 2x2 и шагом 2, уменьшает размер в 2 раза
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Второй сверточный слой: 32 входных канала, 64 выходных фильтра, ядро 3x3, паддинг 1
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # Полносвязный слой (fc1): входной размер - 64 канала * 8 * 8 (размер после 2 пуллингов),
        # выход 128 нейронов
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        # Полносвязный слой (fc2): вход 128 нейронов, выход 10 классов (CIFAR-10)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        # F.relu — функция активации ReLU, добавляет нелинейность
        # Сначала свертка conv1 + ReLU + пуллинг (уменьшение размера вдвое)
        x = self.pool(torch.relu(self.conv1(x)))
        # Далее свертка conv2 + ReLU + пуллинг (еще раз уменьшаем размер)
        x = self.pool(torch.relu(self.conv2(x)))
        # x.view преобразует тензор в плоский вид (-1 — размер батча, 64*8*8 — размер признаков)
        x = x.view(-1, 64 * 8 * 8)
        # Полносвязный слой с ReLU
        x = torch.relu(self.fc1(x))
        # Финальный слой с логарифмом вероятностей по классам (log_softmax)
        x = torch.log_softmax(self.fc2(x), dim=1)
        return x


In [3]:
# Загрузка и предобработка датасета CIFAR-10
transform = transforms.Compose([
    transforms.ToTensor(),  # преобразует PIL-изображение в тензор (значения [0,1])
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    # нормализация каждого канала (R,G,B): вычитаем среднее 0.5 и делим на std 0.5
    # для ускорения и стабилизации обучения
])

# train=True — загружаем тренировочную часть датасета, download=True — скачиваем, если нет локально
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# DataLoader — удобный класс для загрузки данных пакетами (batch_size=100), shuffle=True перемешивает данные
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

# Аналогично для тестового набора
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Классы CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170M/170M [00:12<00:00, 13.1MB/s]


In [4]:


# Инициализация сети, функции потерь и оптимизатора
net = SimpleCNN()
criterion = nn.CrossEntropyLoss()
# CrossEntropyLoss — стандартная функция потерь для задач классификации, учитывает softmax
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# SGD — стохастический градиентный спуск с learning rate=0.001 и моментумом 0.9 (ускоряет обучение)

# Обучение сети
num_epochs = 10
train_losses = []
test_losses = []

for epoch in range(num_epochs):  # цикл по эпохам (полный проход по датасету)
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data  # получаем входные изображения и метки классов
        optimizer.zero_grad()  # обнуляем градиенты перед обратным проходом

        outputs = net(inputs)  # прямой проход: считаем предсказания сети
        loss = criterion(outputs, labels)  # вычисляем ошибку (потерю)
        loss.backward()  # обратное распространение ошибки — вычисляем градиенты
        optimizer.step()  # обновляем веса модели на основе градиентов

        running_loss += loss.item()  # накапливаем потерю для вывода
        if i % 100 == 99:  # каждые 100 пакетов выводим среднюю потерю
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

# Тестирование модели на тестовом наборе
correct = 0
total = 0
with torch.no_grad():  # отключаем вычисление градиентов, чтобы сэкономить память и время
    for data in testloader:
        inputs, labels = data
        outputs = net(inputs)  # предсказания модели
        _, predicted = torch.max(outputs.data, 1)
        # torch.max выбирает класс с максимальной вероятностью по dim=1 (по классам)
        total += labels.size(0)  # суммируем общее количество примеров
        correct += (predicted == labels).sum().item()
        # считаем количество правильных предсказаний

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')
# Выводим точность — процент правильных предсказаний на тесте


[Epoch 1, Batch 100] loss: 2.294
[Epoch 1, Batch 200] loss: 2.261
[Epoch 1, Batch 300] loss: 2.190
[Epoch 1, Batch 400] loss: 2.071
[Epoch 1, Batch 500] loss: 1.992
[Epoch 2, Batch 100] loss: 1.909
[Epoch 2, Batch 200] loss: 1.854
[Epoch 2, Batch 300] loss: 1.803
[Epoch 2, Batch 400] loss: 1.768
[Epoch 2, Batch 500] loss: 1.734
[Epoch 3, Batch 100] loss: 1.710
[Epoch 3, Batch 200] loss: 1.667
[Epoch 3, Batch 300] loss: 1.627
[Epoch 3, Batch 400] loss: 1.605
[Epoch 3, Batch 500] loss: 1.582
[Epoch 4, Batch 100] loss: 1.544
[Epoch 4, Batch 200] loss: 1.525
[Epoch 4, Batch 300] loss: 1.507
[Epoch 4, Batch 400] loss: 1.469
[Epoch 4, Batch 500] loss: 1.459
[Epoch 5, Batch 100] loss: 1.421
[Epoch 5, Batch 200] loss: 1.412
[Epoch 5, Batch 300] loss: 1.401
[Epoch 5, Batch 400] loss: 1.401
[Epoch 5, Batch 500] loss: 1.376
[Epoch 6, Batch 100] loss: 1.355
[Epoch 6, Batch 200] loss: 1.339
[Epoch 6, Batch 300] loss: 1.342
[Epoch 6, Batch 400] loss: 1.325
[Epoch 6, Batch 500] loss: 1.310
[Epoch 7, 